In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import xlwings as xl
import regex as re

In [3]:
ten_dang_nhap = ""
mat_khau = ""

#------------------------------------------------------------------------------------

driver = webdriver.Edge()
driver.get(r"https://careerbuilder.vn/vi/employers/login")
wait = WebDriverWait(driver, 10)
driver.find_element_by_xpath(r'//*[@id="frmLogin"]/div[1]/div[2]/input').send_keys(ten_dang_nhap)
driver.find_element_by_xpath(r'//*[@id="frmLogin"]/div[2]/div[2]/input').send_keys(mat_khau)


#sign in button
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#frmLogin>div.user-action>div>button'))).click()

#Quản lý ứng viên
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body>main>section.employer-navbar-2-1>div>div.main-wrap>div.left-wrap>ul>li:nth-child(3)>a'))).click()

#Hồ sơ ứng tuyển
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body>main>section.manage-candidates-resume-applied.cb-section.bg-manage>div>div>div.main-tabslet>ul>li.active>a'))).click()

#CV links
time.sleep(15)
name_links = []
while True:

    names = driver.find_elements_by_class_name('name')
    name_links.extend([name.get_attribute('href') for name in names][2:])
    try:
        next = driver.find_element_by_xpath('//em[@class = "mdi mdi-chevron-right"]')
        next.click()
        time.sleep(5)
    except:
        print("done")
        break


#open link in new tab, get content then close the new tab
contents = []
for i in range(len(name_links)):
    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[1])
    CV_Url = name_links[i]
    driver.get(CV_Url)
    print(i)
    time.sleep(10)
    content_1 = driver.find_element_by_class_name("breadcrumb-gotopage").text
    content_2 = driver.find_element_by_class_name("view-resume-head").text
    content_3 = driver.find_element_by_class_name("view-resume-body").text
    content_4 = driver.find_element_by_class_name("row").text
    content_5 = driver.find_element_by_class_name("job-information").text
    contents.append(content_1 +'\n'+ content_2 +'\n'+ content_3 +'\n'+ content_4 +'\n'+ content_5)
    time.sleep(5)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(5)



#empty dataframe 
header = 'Stt	Ngày nhận hồ sơ	Ngày nhận HS 	Tháng nhận HS	Năm nhận HS	Công ty	Đơn vị / Chi nhánh	Khối/TT	Phòng ban/Bộ phận	Vị trí	Họ và tên	Ngày tháng năm sinh	Số điện thoại	Email	Tuổi	Giới tính	Tốt nghiệp trường	Trình độ cao nhất	Địa chỉ	Nguồn	Hình thức	Loại kênh	Kênh TD	KQ hồ sơ'
header = header.split('\t')
df = pd.DataFrame(columns = header)
df1 = pd.DataFrame(columns = header)
header_index = [1,9,10,11,12,13,15,17,18,-1]
fill = [header[i] for i in header_index]

#regex
applying_date = r"Ngày nộp:\n(.+)\n"
position = r'Hồ Sơ Ứng Tuyển\n(.+)\n'
name = 'Ứng viên:\n(.+)\n'
year_of_birth = 'Ngày sinh:\n\d\d/\d\d\/(\d\d\d\d)\n'
phone = r'ĐTDĐ :\n(.+)\n'
email = r'Email :\n(.+)\n'
sex = r'Giới tính:\n(\w+) '
degree = r'Bằng cấp cao nhất:\n(.+)\n'
home = r'Phố:\n(.+)\n'
status = r"Trạng thái:\n(.+)\n"


#fill the dataframe
ls = [applying_date, position, name, year_of_birth, phone, email, sex, degree, home, status]

for i in contents:
    regex_result = []
    for j in ls:
        a = re.findall(j, i, flags = re.UNICODE)
        regex_result.append(a)

    for e in range(len(fill)):
        try:
            df["%s"%fill[e]] = regex_result[e]
            if e == 4:
                df["%s"%fill[e]] = "'" + regex_result[e][0]
        except ValueError:
            df["%s"%fill[e]] = 'zzzzzzzzzz'
    df1 = df1.append(df)

df1["Nguồn"] = "Bên ngoài"
df1["Hình thức"] = "Online"
df1["Loại kênh"] = "Website"
df1["Kênh TD"] = "CareerBuilder"

df1['Ngày nhận hồ sơ'] = pd.to_datetime(df1['Ngày nhận hồ sơ'], format = r'%d-%m-%Y')


df1.iloc[:,[2]] = df1['Ngày nhận hồ sơ'].dt.day
df1.iloc[:,[3]] = df1['Ngày nhận hồ sơ'].dt.month
df1.iloc[:,[4]] = df1['Ngày nhận hồ sơ'].dt.year

try:
    df1 = df1.astype({"Ngày tháng năm sinh": int})
    df1['Tuổi'] = df1['Năm nhận HS'] - df1['Ngày tháng năm sinh']
except: 
    df1['Tuổi'] = 'zzzzzzz'

#send data to excel
book = xl.books.open(r"D:\Masscom\Data\2022_Data Tuyen dung.xlsx")
sheet = book.sheets("CareerBuilder")
sheet["A2"].options(pd.DataFrame,index = False, header = False).value = df1

C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(r'//*[@id="frmLogin"]/div[1]/div[2]/input').send_keys(ten_dang_nhap)
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(r'//*[@id="frmLogin"]/div[2]/div[2]/input').send_keys(mat_khau)
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  names = driver.find_elements_by_class_name('name')
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next = driver.find_element_by_xpath('//em[@class = "mdi mdi-chevron-right"]')


done
0


C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:47: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_1 = driver.find_element_by_class_name("breadcrumb-gotopage").text
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_2 = driver.find_element_by_class_name("view-resume-head").text
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_3 = driver.find_element_by_class_name("view-resume-body").text
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/3644658283.py:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_4 = driver.find_element_by_class_name("row").text
C:\Users\Admin\AppData\Local\Temp/ipykernel_18844/36446582

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [12]:
driver.quit()